# Pushing excel tables into SQL

This notebook is aimed to retrieve the scattered data from different sheets contained in a google spreadshee

In [ ]:
import pandas as pd
import json
from pybliometrics.scopus import AbstractRetrieval, AuthorRetrieval, AffiliationRetrieval
import warnings
import datetime
import sqlalchemy

In [ ]:
## Load configuration from json file 
# Run this code only if the excel file is in googlespreadsheets
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

## Initializing SQL

In [ ]:
schema = config["schema"]
host = config["host"]
user = config["user"]
password = config["password"]
port = config["port"]

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
connection_string

## Importing and functions

In [ ]:
#importing and opening excel file from a google spreadhsheets. Run this only if you are using google spreadsheets
url_id = config['url_id']
url_base = "https://docs.google.com/spreadsheets/d/"
url_e = "export/format=xlsx"
url_params = url_base+url_id+url_e

def table_import(url_params, tab_name):
    file = pd.ExcelFile(url_params)  
    with pd.ExcelFile(file) as xls:  
        df = pd.read_excel(xls, tab_name, header=0)
    #ignoring warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning, message=".*utcnow.*")
    now = datetime.datetime.utcnow()
    return df

def table_import2(xls):
    with pd.ExcelFile(file) as xls:  
        df = pd.read_excel(xls, tab_name, header=0)
    return df

In [ ]:
# Pathto the Excel file if you are not using google spreadhseets
file_path = "database/excel_tables/master_table_copy.xlsx"  # Replace with your actual file path

# Load the Excel file
file = pd.ExcelFile(file_path)

The function below is to change the arragement and format of the different tables

In [ ]:
def df_format(df):

    #removing useless columns for analysis
    columns_to_drop = ['AI (yes/no)', 'Sustainability (yes/no)']
    df = df.drop(columns=columns_to_drop)

    # renaming the columns
    columns_to_rename = {'Authors': 'authors',
                         'EID': 'eid',
                         'Title': 'title',
                         'Year': 'year_pub',
                         'SDG': 'sdg',
                         "Abstract": "abstract",
                         'Source title': 'journal',
                         'Cited by': 'cited_by',
                         'core topic': 'core_topic',
                         'DOI': 'doi',
                         'role of AI': 'role_of_AI',
                         'AI (yes/no)': 'is_AI?',
                         'sustainability definition': 'sustainability_definition',
                         'Sustainability (yes/no)': 'is_sustainability?',
                         'Method (1) vs. study object (2)': 'AI_scope',
                         'Means (1) vs. end (2)': 'means_vs_ends',
                         'AI as buzzword? (0/1)': 'AI_as_buzzword',
                         'Sus_lvl': 'sustainability_conceptualization',
                         'empirical/conceptual/review': 'article_type',
                         'spatial scale': 'spatial_scale',
                         'snapshot in time vs. longitudinal study': 'temporal_scope',
                         'temporal scale (past, present, future)': 'temporal_scale',
                         'qualitative/quantitative/mixed methods': 'methodology',
                         'location of the study (country)': 'country_study',
                         'dataset used': 'dataset_used',
                         'first_author_country': 'country_author',
                         'policy recommendations (1/0)': 'policy_recommendations'
                        }
    df = df.rename(columns= columns_to_rename)

    df["sustainability_conceptualization"] = df["sustainability_conceptualization"].str.lower()
    df["sustainability_conceptualization"] = df["sustainability_conceptualization"].str.lower()
    df["sustainability_conceptualization"] = df["sustainability_conceptualization"].str.strip()
    
    df["article_type"] = df["article_type"].str.lower()
    df["article_type"] = df["article_type"].str.lower()
    df["article_type"] = df["article_type"].str.strip()

    return df

## Inspection (Importing final table)

In [ ]:
df_raw = table_import(url_params, "final_table") #only use if you are using gspreadsheets
#or
#df_raw = table_import2(file)
df_raw.info()

In [ ]:
#applying the function format
df = df_format(df_raw)
df.info()

In [ ]:
#veryfing that the three types of article exist
df.loc[(df["article_type"]=="empirical"), "methodology"].unique()

In [ ]:
#checking how many empirical papers are
df.loc[(df["article_type"]=="empirical"),].count()

## <span style="background-color:green">Articles dataframe</span> 

Now, the next step is to create a table or dataframe with the articles metadata

### Creating dataframe

In [ ]:
metadata_df = df.loc[:,["eid", "title", "authors", "year_pub",
                        "sdg", "journal", "cited_by", "doi"]]


metadata_df['sdg'] = metadata_df['sdg'].astype('str')

metadata_df.info()

### Pushing it to SQL

In [ ]:
metadata_df.to_sql('articles',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:purple">Empirical data dataframe</span> 

Because the empirical articles were coded different and contain specific columns, a specific dataframe was created

In [ ]:
columns_to_boolean = ["AI_as_buzzword", "policy_recommendations"]
df[columns_to_boolean] = df[columns_to_boolean].astype('bool')

empirical_df = df.loc[:, ["eid", "abstract", "AI_scope", "AI_as_buzzword", "core_topic",
                          "means_vs_ends", "sustainability_definition", "spatial_scale",
                          "temporal_scope", "temporal_scale", "methodology", "country_study",
                           "dataset_used", "country_author", "policy_recommendations"]]

empirical_df.info()

In [ ]:
empirical_df.to_sql("empirical_data",
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:blue">Type of article and suslvl dataframe</span> 

In [ ]:
type_article = df[["eid","article_type"]]
type_article.to_sql('type_of_article',
                  if_exists='append',
                  con=connection_string,
                  index=False)

In [ ]:
suslvl_article = df[["eid","sustainability_conceptualization"]]
suslvl_article.to_sql('sus_lvl',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:red">Algorithms-articles dataframe</span>

### Function to separate values in the same cell

In [ ]:
def separating(df, old_column, new_column):
    df[new_column] = df[old_column].str.split(';')
    df = df.explode(new_column)
    df = df.drop(columns=old_column).reset_index(drop=True)
    df[new_column] = df[new_column].str.strip()
    df[new_column] = df[new_column].str.lower()
    return df

### Creating dataframe

In [ ]:
# aral means ar(ticle)al(gorithm)
aral_df = df.loc[:, ["eid", "algorithms"]]
aral_df = separating(aral_df, "algorithms", "algorithms_used")
aral_df = aral_df.sort_values(by="algorithms_used", ascending=True).reset_index(drop=True)
aral_df = aral_df.rename(columns= {"algorithms_used": "algorithms"})
aral_df.info()

### Pushing it to SQL

In [ ]:
aral_df.to_sql('articles_algorithms',
                if_exists='append',
                con=connection_string,
                index=False)

## <span style="background-color:brown">Algorithms, type of AI and tasks dataframes</span> 

### Importing sheet from drive

In [ ]:
names_df = table_import(url_params, "algorithms abbreviations")
names_df["Algorithm name"] = names_df["Algorithm name"].str.lower()
names_df["Abbreviations"] = names_df["Abbreviations"].str.lower()
names_df["Abbreviations"] = names_df["Abbreviations"].str.strip()
names_df

### algorithms_list table

In [ ]:
## algorithms dataframe
algo_df = names_df.drop_duplicates(subset="Abbreviations")[["Abbreviations", "Algorithm name"]]

columns_to_rename = {"Abbreviations": "algorithms",
                     "Algorithm name": "algorithm_name"
                    }
algo_df = algo_df.rename(columns= columns_to_rename)

algo_df.info()

In [ ]:
algo_df.to_sql('algorithms_list',
                if_exists='append',
                con=connection_string,
                index=False)

### ai_type_list_table

In [ ]:
aitype = separating(names_df, "Type of AI", "ai_type")[["ai_type"]]
aitype = aitype.drop_duplicates(subset="ai_type")
aitype = aitype.reset_index(drop=True)
aitype

In [ ]:
aitype.to_sql('ai_type_list',
                if_exists='append',
                con=connection_string,
                index=False)

### task_type_list

In [ ]:
tasktype = separating(names_df, "Type of task", "task_type")[["task_type"]]
tasktype = tasktype.drop_duplicates(subset="task_type")
tasktype = tasktype.reset_index(drop=True)

tasktype.info()

In [ ]:
tasktype.to_sql('task_type_list',
                if_exists='append',
                con=connection_string,
                index=False)

### algorithms_aitype

In [ ]:
algo_ai = separating(names_df, "Type of AI", "ai_type")[["Abbreviations","ai_type"]]
algo_ai = algo_ai.rename(columns= {"Abbreviations":"algorithms"})
algo_ai

In [ ]:
algo_ai.to_sql('algorithms_aitype',
                if_exists='append',
                con=connection_string,
                index=False)

### algorithms_tasktype

In [ ]:
algo_task = separating(names_df, "Type of task", "task_type")[["Abbreviations","task_type"]]
algo_task = algo_task.rename(columns= {"Abbreviations":"algorithms"})
algo_task

In [ ]:
algo_task.to_sql('algorithms_tasktype',
                if_exists='append',
                con=connection_string,
                index=False)

## <span style="background-color:turquoise">Role of AI</span> 

In [ ]:
role_ai = df[["eid","role_of_AI"]]
role_ai.to_sql('role_AI',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:violet">Clusters revcon dataframe</span>

In [ ]:
#importing and opening file from google spreadsheets
groups_df = table_import(url_params, "words_clusters_revcon")
groups_df['cluster_number'] = groups_df['cluster_number'].astype('str')
groups_df.info()

In [ ]:
#merging with metadata df
merged_df = pd.merge(metadata_df, groups_df, on='eid', how='inner')
groups_3_df = merged_df[["eid", "words", "cluster_number", "cluster_name", "article_file"]]
groups_3_df.info()

In [ ]:
groups_3_df.to_sql('clusters_revcon',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:violet">Clusters dataframe</span>

In [ ]:
#importing and opening file from google spreadsheets
groups_df = table_import(url_params, "words_clusters")
groups_df['cluster_number'] = groups_df['cluster_number'].astype('str')
groups_df.info()

In [ ]:
#merging with metadata df
merged_df = pd.merge(metadata_df, groups_df, on='eid', how='inner')
groups_2_df = merged_df[["eid", "words", "cluster_number", "cluster_name", "article_file"]]
groups_2_df.info()

In [ ]:
groups_2_df.to_sql('clusters',
                  if_exists='append',
                  con=connection_string,
                  index=False)

## <span style="background-color:gray">Authors dataframe</span> 

In [ ]:
#creating the function
def articles_info(eids):
    arts = []
    for eid in eids:
        #applying pybliometrics
        arts.append(AbstractRetrieval(eid, view='FULL'))
    return arts

In [ ]:
#extracting all the eid's and scopus id's
eids = df["EID"].tolist()
#calling the function, make sure to have your API key running
articles = articles_info(eids)

In [ ]:
#creating dictionary
authors = {
            'article_eid': [],
            'author_id': [],
            'author_name': [],
            'affiliation_country': [],
            'position in article': []
            }
#iterating over each element in the articles list
for article in articles:
    #iterating over each element in authorgroup
    for i in range(len(article.authorgroup)):
        authors['article_eid'].append(article.eid)
        authors['author_id'].append(article.authorgroup[i].auid)
        first_name = article.authorgroup[i].given_name
        last_name = article.authorgroup[i].surname
        authors['author_name'].append(f'{first_name} {last_name}')
        authors['affiliation_country'].append(article.authorgroup[i].country)
        authors['position in article'].append(str(i+1))

authors_df = pd.DataFrame(authors)

In [ ]:
authors_df.loc[authors_df["position in article"]=="1",]